In [1]:
from pathlib import Path
import sys

In [2]:
notebook_dir = Path.cwd()

if 'notebooks' in str(notebook_dir):
    project_root = notebook_dir.parent.parent
else:
    # If already at project root, use current directory
    project_root = notebook_dir

# Add src/macronome to Python path
sys.path.insert(0, str(project_root / "src"))
sys.path.insert(0, str(project_root))

In [3]:
from macronome.ai.prompts.manager import PromptManager

In [4]:
chat_prompt = PromptManager.get_prompt("chat_router", chat_history=[], message="")
print(chat_prompt)


You are analyzing a user's chat message in a meal recommendation app. Your job is to determine what the user wants to do.


**User's Message:**


**Your Task:**
Analyze the message and classify it into ONE of these actions:

1. **ADD_CONSTRAINT** - User wants to add or modify meal constraints
   Examples:
   - "I want something vegan"
   - "Make it under 700 calories"
   - "I'm allergic to peanuts"
   - "Something spicy"
   - "Quick meal, under 30 minutes"
   - "I prefer Italian food"

2. **START_RECOMMENDATION** - User explicitly requests a meal recommendation
   Examples:
   - "Recommend me a meal"
   - "What should I cook?"
   - "Find me a recipe"
   - "Give me a meal suggestion"
   - "I'm ready to see options"

3. **GENERAL_CHAT** - User has questions, needs help, or general conversation
   Examples:
   - "How does this app work?"
   - "What can I do here?"
   - "Can you explain the filters?"
   - "Hello"
   - "Thanks!"

**Important Guidelines:**
- If the user mentions ANY constra

In [5]:
from experiments.utils.event_loader import EventLoader
import logging
import nest_asyncio
from macronome.ai.workflows.workflow_registry import WorkflowRegistry

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()


/Users/ravirai/GitHub/macronome/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Testing Chat Workflow

In [6]:
"""
This playground is used to test the WorkflowRegistry and the workflows themselves.
"""

event = EventLoader.load_event(event_key="chat_workflow_start_recommendation")

In [7]:
workflow = WorkflowRegistry.CHAT.value()  # Use CHAT, not ChatWorkflow

In [17]:
output = workflow.run(event)


INFO:root:Starting node: ChatRouter
INFO:root:Finished node: ChatRouter


[ChatRouter] Task context: event=ChatRequest(message="I'm ready! Can you recommend me a meal based on my preferences?", chat_session_id='test-session-123', user_id='test-user-456', chat_history=[{'role': 'user', 'content': 'Hi, can you add a constraint for low-carb meals?'}, {'role': 'assistant', 'content': "I've added low-carb to your dietary restrictions! This means I can help you discover meals that better align with your preferences."}, {'role': 'user', 'content': 'Actually, make that under 500 calories too'}, {'role': 'assistant', 'content': "I've updated your calorie target to 500. Your preferences are now set to low-carb and under 500 calories!"}], user_preferences={'default_constraints': {'calories': 500, 'macros': {'carbs': 50, 'protein': 60, 'fat': 25}, 'prepTime': 30}, 'dietary_restrictions': ['vegan', 'low-carb'], 'custom_constraints': {'spicy': False, 'cuisine': 'italian'}, 'disliked_ingredients': ['mushrooms', 'olives'], 'favorite_cuisines': ['italian', 'mediterranean']})

INFO:root:Starting node: ResponseGenerator


[ChatRouter] Available nodes: ['ChatRouter', 'ConstraintParser', 'ResponseGenerator']
[ChatRouter] Routing to ResponseGenerator
[ChatRouter] Created next node instance: ResponseGenerator


INFO:root:Finished node: ResponseGenerator


In [18]:
output.model_dump()

{'event': {'message': "I'm ready! Can you recommend me a meal based on my preferences?",
  'chat_session_id': 'test-session-123',
  'user_id': 'test-user-456',
  'chat_history': [{'role': 'user',
    'content': 'Hi, can you add a constraint for low-carb meals?'},
   {'role': 'assistant',
    'content': "I've added low-carb to your dietary restrictions! This means I can help you discover meals that better align with your preferences."},
   {'role': 'user', 'content': 'Actually, make that under 500 calories too'},
   {'role': 'assistant',
    'content': "I've updated your calorie target to 500. Your preferences are now set to low-carb and under 500 calories!"}],
  'user_preferences': {'default_constraints': {'calories': 500,
    'macros': {'carbs': 50, 'protein': 60, 'fat': 25},
    'prepTime': 30},
   'dietary_restrictions': ['vegan', 'low-carb'],
   'custom_constraints': {'spicy': False, 'cuisine': 'italian'},
   'disliked_ingredients': ['mushrooms', 'olives'],
   'favorite_cuisines': 

## Testing Pantry Scanner Workflow

In [6]:
event_data = EventLoader.load_event(event_key="pantry_scanner")

In [7]:
from pathlib import Path
from PIL import Image

In [8]:
# Load the image from the path (resolve relative to project root)
image_path = project_root / event_data["image_path"]
print(f"Loading image from: {image_path}")
print(f"Image exists: {image_path.exists()}")

pil_image = Image.open(image_path)
print(f"Image size: {pil_image.size}, mode: {pil_image.mode}")

# Convert to RGB if needed
if pil_image.mode != "RGB":
    pil_image = pil_image.convert("RGB")

# Create the workflow request (matching PantryScanRequest schema)
event = {
    "image": pil_image,  # PIL Image object, not a path string
    "conf_threshold": event_data.get("conf_threshold", 0.25),
    "crop_padding": event_data.get("crop_padding", 10)
}

Loading image from: /Users/ravirai/GitHub/macronome/data/processed/yolo_format/test/images/img_0000.jpg
Image exists: True
Image size: (416, 416), mode: RGB


In [9]:
workflow = WorkflowRegistry.PANTRY_SCANNER.value()

In [10]:
output = workflow.run(event)

INFO:root:Starting node: DetectionNode
INFO:macronome.ai.workflows.pantry_scanner_nodes.detection_node:🔍 Detecting items in pantry image via API endpoint...
INFO:macronome.ai.workflows.pantry_scanner_nodes.detection_node:   Detected 6 items
INFO:root:Finished node: DetectionNode
INFO:root:Starting node: CroppingNode
INFO:macronome.ai.workflows.pantry_scanner_nodes.cropping_node:✂️  Cropping 6 items...
INFO:macronome.ai.workflows.pantry_scanner_nodes.cropping_node:   Cropped 6 images
INFO:root:Finished node: CroppingNode
INFO:root:Starting node: ClassificationNode
INFO:macronome.ai.workflows.pantry_scanner_nodes.classification_node:🥘 Classifying 6 items...
18:27:56 - LiteLLM:INFO: utils.py:3416 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai
18:27:57 - LiteLLM:INFO: utils.py:1301 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
18:27:57 - LiteLLM:I


✅ Classification Results (6 items):
  1. mott's apple juice (confidence: 0.92)
  2. crack berry (confidence: 0.90)
  3. yuengling lager (confidence: 0.84)
  4. diet mountain dew (confidence: 0.46)
  5. unknown (confidence: 0.32)
  6. unknown (confidence: 0.29)



## Testing Meal Recommendation Workflow

In [6]:
event_data = EventLoader.load_event(event_key="meal_recommendation_1")

In [7]:
workflow = WorkflowRegistry.MEAL_RECOMMENDER.value()

In [8]:
output = workflow.run(event_data)

INFO:root:Starting node: NormalizeNode
INFO:root:Finished node: NormalizeNode
INFO:root:Starting node: PlanningAgent
INFO:root:Finished node: PlanningAgent
INFO:root:Starting node: RetrievalNode
INFO:macronome.ai.workflows.meal_recommender_workflow_nodes.retrieval_node:Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:macronome.ai.workflows.meal_recommender_workflow_nodes.retrieval_node:Connecting to Qdrant at https://9820b6b8-2ef8-45e6-8ed5-484c8c1a7ae7.us-east-1-1.aws.cloud.qdrant.io:6333
INFO:macronome.ai.workflows.meal_recommender_workflow_nodes.retrieval_node:Connected to Qdrant collection 'recipes' with 500000 recipes
INFO:macronome.ai.workflows.meal_recommender_workflow_nodes.retrieval_node:Retrieving recipes with query: 'Chicken Lunch with Chicken Breast: 1 chicke

In [9]:
output.model_dump()

{'event': {'user_query': 'recommend a meal for me, ideally include chicken so i can finish it before it gets old',
  'constraints': {'calories': 800,
   'macros': {'carbs': 100, 'protein': 55, 'fat': 20},
   'diet': None,
   'allergies': ['nuts'],
   'prep_time': 15,
   'meal_type': 'lunch',
   'custom_constraints': {'spicy': False, 'cuisine': 'italian'}},
  'pantry_items': [{'name': 'chicken breast',
    'category': 'protein',
    'confirmed': True},
   {'name': 'tomatoes', 'category': 'vegetable', 'confirmed': True},
   {'name': 'olive oil', 'category': 'oil', 'confirmed': True},
   {'name': 'garlic', 'category': 'spice', 'confirmed': True},
   {'name': 'whole wheat bread', 'category': 'grain', 'confirmed': True}],
  'chat_history': []},
 'nodes': {'NormalizeNode': {'model_output': {'calorie_range': [750, 850],
    'macro_targets': {'carbs': 100, 'protein': 55, 'fat': 20},
    'diet_type': None,
    'excluded_ingredients': {'nuts'},
    'prep_time_max': 15,
    'custom_constraints': 